In [ ]:
# Voila Notebook: Map Visualization for Enriched Social Media Data
import pandas as pd
import matplotlib.pyplot as plt
from ipyleaflet import Map, Marker, MarkerCluster, Heatmap, LegendControl, basemaps, basemap_to_tiles, FullScreenControl
from ipywidgets import Output, VBox, HTML, Dropdown, Button, HBox, DatePicker, IntSlider, Label, RadioButtons, ToggleButtons, Layout, Accordion
from IPython.display import display
from fpdf import FPDF
import datetime
import io

guide = Accordion(children=[
    HTML("<b>Step 1:</b> Upload your CSV file containing 'sentiment', 'location', 'timestamp', 'latitude', and 'longitude'."),
    HTML("<b>Step 2:</b> Use the filters to narrow down sentiment types, location, or date range."),
    HTML("<b>Step 3:</b> Click <code>Apply Filters</code> to render the map with heatmaps and sentiment markers."),
    HTML("<b>Step 4:</b> Explore the map. You can zoom in, click on markers, or switch map themes."),
    HTML("<b>Step 5:</b> Export your filtered data to PDF or CSV for offline use or reporting.")
])
guide.set_title(0, '📂 Upload Data')
guide.set_title(1, '🎛️ Set Filters')
guide.set_title(2, '✅ Apply Filters')
guide.set_title(3, '🗺️ Explore Map')
guide.set_title(4, '📤 Export & Save')

# === 状态栏输出 ===
status_output = Output()
map_output = Output(layout=Layout(height='700px'))  # 设置更大高度
chart_output = Output()

# === 过滤器控件 ===
sentiment_filter = Dropdown(options=['All', 'positive', 'neutral', 'negative'], description='Sentiment:')
location_filter = Dropdown(options=['All'], description='Location:')
date_start = DatePicker(description='From:', value=datetime.date(2025, 1, 1))
date_end = DatePicker(description='To:', value=datetime.date(2025, 12, 31))
heat_radius = IntSlider(description='Heat Radius', min=5, max=50, step=1, value=20)
map_theme = RadioButtons(options=['Default', 'Dark', 'Satellite'], description='Theme:')
lang_toggle = ToggleButtons(options=['English', '中文'], description='Language:')
apply_button = Button(description='Apply Filters', button_style='success')
export_button = Button(description='Export PDF', button_style='info')
export_csv_button = Button(description='Export CSV', button_style='warning')
reset_button = Button(description='Reset Filters', button_style='danger')
save_button = Button(description='Save View Settings', button_style='primary')

In [ ]:
def load_and_render_map(
    file_path='../data/enriched_user_data.csv',
    sentiment='All',
    location='All',
    start_date=None,
    end_date=None,
    radius=20,
    theme='Default'
):
    status_output.clear_output()
    map_output.clear_output()
    chart_output.clear_output()

    try:
        df = pd.read_csv(file_path)
        print("✅ load df successfully!")
    except Exception as e:
        with status_output:
            print(f"❌ Failed to load file: {str(e)}")
        return

    if not {'latitude', 'longitude', 'text', 'sentiment', 'location', 'timestamp'}.issubset(df.columns):
        with status_output:
            print("⚠️ Required columns missing: latitude, longitude, text, sentiment, location, timestamp")
        return

    df['timestamp'] = pd.to_datetime(df['timestamp'])

    if sentiment != 'All':
        df = df[df['sentiment'] == sentiment]
    if location != 'All':
        df = df[df['location'] == location]
    if start_date and end_date:
        df = df[(df['timestamp'].dt.date >= start_date) & (df['timestamp'].dt.date <= end_date)]

    if df.empty:
        with status_output:
            print("⚠️ No data matches the filters.")
        return

    # ✅ 仅使用包含有效坐标的数据绘制地图
    df_map = df.dropna(subset=["latitude", "longitude"])

    if df_map.empty:
        with status_output:
            print("⚠️ 所有数据点缺失地理坐标，无法在地图上显示。")
        return

    with map_output:
        center = (df_map['latitude'].mean(), df_map['longitude'].mean())
        if theme == 'Dark':
            tiles = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
        elif theme == 'Satellite':
            tiles = basemap_to_tiles(basemaps.Esri.WorldImagery)
        else:
            tiles = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)

        m = Map(center=center, zoom=2, layout=Layout(height='650px'))
        m.add_layer(tiles)
        m.add_control(FullScreenControl())  # 添加全屏按钮


        markers = []
        for _, row in df_map.iterrows():
            tooltip = f"<b>Sentiment:</b> {row['sentiment']}<br><b>Text:</b> {row['text']}"
            marker = Marker(location=(row['latitude'], row['longitude']), draggable=False)
            marker.popup = HTML(tooltip)
            markers.append(marker)
        m.add_layer(MarkerCluster(markers=markers))

        heat_data = [[row['latitude'], row['longitude']] for _, row in df_map.iterrows()]
        # heat_layer = Heatmap(locations=heat_data, radius=radius)
        max_density = min(len(heat_data) / 20, 10)  # 根据数据量自动调整
        heat_layer = Heatmap(
            locations=heat_data,
            radius=radius,
            max=max_density
        )
        m.add_layer(heat_layer)

        legend = LegendControl({
            "Positive": "green",
            "Neutral": "blue",
            "Negative": "red"
        }, name="Sentiment Legend", position="bottomright")
        m.add_control(legend)

        display(m)

    # 更新 location 下拉菜单
    loc_options = ['All'] + sorted(df['location'].dropna().unique().tolist())
    location_filter.options = loc_options

    with chart_output:
        sentiment_counts = df['sentiment'].value_counts()
        plt.figure(figsize=(5,3))
        sentiment_counts.plot(kind='bar', color=['green','blue','red'])
        plt.title("Sentiment Distribution")
        plt.xlabel("Sentiment")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.show()

    return df


In [ ]:
def apply_filters(btn):
    global latest_df
    latest_df = load_and_render_map(
        sentiment=sentiment_filter.value,
        location=location_filter.value,
        start_date=date_start.value,
        end_date=date_end.value,
        radius=heat_radius.value,
        theme=map_theme.value
    )

def export_filtered_data(btn):
    if latest_df is not None:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Exported: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)

        for i, row in latest_df.iterrows():
            line = f"{row['timestamp']} | {row['location']} | {row['sentiment']} | {row['text'][:50]}..."
            pdf.cell(200, 10, txt=line, ln=True)
            if i >= 20:
                pdf.cell(200, 10, txt="... (truncated)", ln=True)
                break

        pdf.output("filtered_data_export.pdf")
        with status_output:
            print("✅ Exported to filtered_data_export.pdf")

def export_csv(btn):
    if latest_df is not None:
        latest_df.to_csv("filtered_data_export.csv", index=False)
        with status_output:
            print("✅ Exported to filtered_data_export.csv")

def reset_filters(btn):
    sentiment_filter.value = 'All'
    location_filter.value = 'All'
    date_start.value = datetime.date(2025, 1, 1)
    date_end.value = datetime.date(2025, 12, 31)
    heat_radius.value = 20
    map_theme.value = 'Default'
    apply_filters(None)

def save_view_settings(btn):
    with status_output:
        print("💾 Current view settings saved:")
        print(f"• Sentiment: {sentiment_filter.value}")
        print(f"• Location: {location_filter.value}")
        print(f"• Date range: {date_start.value} → {date_end.value}")
        print(f"• Heat Radius: {heat_radius.value}")
        print(f"• Theme: {map_theme.value}")

apply_button.on_click(apply_filters)
export_button.on_click(export_filtered_data)
export_csv_button.on_click(export_csv)
reset_button.on_click(reset_filters)
save_button.on_click(save_view_settings)
latest_df = load_and_render_map()


In [ ]:
VBox([
    guide,
    HTML("<h2>🗺️ Visualize Social Media Data on Map</h2>"),
    lang_toggle,
    HBox([sentiment_filter, location_filter], layout=Layout(justify_content='flex-start', gap='20px')),
    HBox([date_start, date_end, heat_radius], layout=Layout(justify_content='flex-start', gap='20px')),
    HBox([map_theme], layout=Layout(justify_content='flex-start', gap='20px')),
    HBox([apply_button, reset_button, export_button, export_csv_button, save_button], layout=Layout(justify_content='flex-start', gap='12px')),
    status_output,
    map_output,
    chart_output
])